In [2]:
import os
MODEL_DIR = '../models'
LOG_DIR = '../logs/02/'
SIZE = (250, 250)
caminho_bases = os.path.join('..', 'bases', 'vazios')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

# Callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard

tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Bmodelweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,
                              verbose=1, min_delta=1e-2, mode='min')

# Model

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop

SIZE = (224, 224)

nuclear_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3),
                         padding='same',
                         activation='relu',
                         input_shape=(*SIZE, 3)),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(32, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.2),
  Conv2D(64, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(256, (3, 3), activation='relu'),
  Flatten(),
  Dense(256, activation='relu'),
 
])

nuclear_model.summary()

In [ ]:
from keras.layers import concatenate
a_in = Input(shape=(*SIZE, 3))
b_in = Input(shape=(*SIZE, 3))

a_feat = nuclear_model(a_in)
b_feat = nuclear_model(b_in)

combined_features = concatenate([a_feat, b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [img_a_in, img_b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['mae'])

# Training

In [23]:
caminho_nvazio_train = os.path.join(caminho_train, 'nvazio')
nvazio_train = [os.path.join(caminho_nvazio_train, arq) for arq in os.listdir(caminho_nvazio_train)]
caminho_vazio_train = os.path.join(caminho_train, 'vazio')
vazio_train = [os.path.join(caminho_vazio_train, arq) for arq in os.listdir(caminho_vazio_train)]


In [29]:
from random import randint

def generate_random_batch(nvazios_list, vazios_list, batch_size=16):
    def get_item(classe):
        if classe == 0:
            return nvazios_list.pop()
        return vazios_list.pop()
    result = []
    for i in range(batch_size):
        classe1 = randint(0, 1)
        item1 = get_item(classe1) 
        classe2 = randint(0, 1)
        item2 = get_item(classe2) 
        result.append((item1, item2, int(classe1 == classe2)))
    return result  

In [30]:
generate_random_batch(nvazio_train, vazio_train)

[('../bases/vazios/train/vazio/5aabcf3d2a87956e81eb1b9b.jpg',
  '../bases/vazios/train/vazio/5aabe7ff2a879503d8050904.jpg',
  1),
 ('../bases/vazios/train/nvazio/5c59d2fee450c23468e2a243.jpg',
  '../bases/vazios/train/nvazio/5c59d555e450c2e7ec0614db.jpg',
  1),
 ('../bases/vazios/train/vazio/5c59d5e1e450c2e88388b738.jpg',
  '../bases/vazios/train/nvazio/5c59d2b9e450c23468e25d5f.jpg',
  0),
 ('../bases/vazios/train/nvazio/5c59d52de450c2e7c7a0ca50.jpg',
  '../bases/vazios/train/vazio/5c644706e450c223d175735d.jpg',
  0),
 ('../bases/vazios/train/nvazio/5c59d3bbe450c2e5a3d2ee70.jpg',
  '../bases/vazios/train/nvazio/5c59d50fe450c2e794932b44.jpg',
  1),
 ('../bases/vazios/train/vazio/5c6446e3e450c223d1755b15.jpg',
  '../bases/vazios/train/vazio/5c59d350e450c23468e2d136.jpg',
  1),
 ('../bases/vazios/train/nvazio/5c59d39ce450c2e57e2fb1f7.jpg',
  '../bases/vazios/train/vazio/5c6446aee450c223d175334f.jpg',
  0),
 ('../bases/vazios/train/nvazio/5c59d2bde450c23468e265ab.jpg',
  '../bases/vazios/t

In [ ]:
def image_triple_generator(image_triples, batch_size):
    while True:
        # loop once per epoch
        num_recs = len(image_triples)
        indices = np.random.permutation(np.arange(num_recs))
        num_batches = num_recs // batch_size
        for bid in range(num_batches):
            # loop once per batch
            batch_indices = indices[bid * batch_size : (bid + 1) * batch_size]
            yield [image_triples[i] for i in batch_indices]
            
triples_batch_gen = image_triple_generator(image_triples, 4)
triples_batch_gen.next()